# Log Creation and Testing

Suppose we have created our chart and we calculated the MACD and RSI. 

In [10]:
import math
import numpy
import random
import decimal
import scipy.linalg
import matplotlib
import numpy.random as nrand
import matplotlib.pyplot as plt
import pandas
from pandas import DataFrame

InitialPrice = 1000
RandomInc = numpy.empty(15000)
RandomPrice = numpy.empty(15000)
decision = numpy.zeros(len(RandomPrice))
RandomPrice[0] = InitialPrice
    
for i in range(15000):
    suddenjump = random.uniform(0,1)
    if suddenjump < 0.1:
        RandomInc[i] = random.uniform(-0.05,0.05)
    else:
        RandomInc[i] = random.uniform(-0.01,0.01)
            
    if i!=14999:
        if RandomPrice[i]+RandomPrice[i]*RandomInc[i] > 50:
            RandomPrice[i+1] = RandomPrice[i]+RandomPrice[i]*RandomInc[i]
        else:   
            RandomPrice[i+1] = RandomPrice[i]-RandomPrice[i]*RandomInc[i]

df = DataFrame(RandomPrice,columns=['Prices'])
df['Perc'] = RandomInc
df.insert(2,'Decision',decision)

exp1 = df['Prices'].ewm(span=12, adjust=False).mean()
exp2 = df['Prices'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
exp3 = macd.ewm(span=9, adjust=False).mean()
df['MACD']=macd
df['Signal Line']=exp3
df['MACD'][0:17]=0
df['Signal Line'][0:26]=0

delta = df['Prices'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
    
df['RSI'] = 100 - (100/(1+rs))
df['RSI'][0:14]=0


Now we'll write our decision mechanism. 

If RSI>70 and MACD Line is smaller than Signal Line this is an indicator for our mechanism to long the stock.
If RSI<30 and MACD Line is bigger than Signal Line this is an indicator for our mechanism to short the stock.

This decision mechanism is very basic and not trustable at all. 
However, since this project is a start to creating a more complex system, it is fine. Our goal is to be able to create a system that creates logs and test how the decision mechanism works. Thus, creating a decision mechanism that is working well will be a problem for the future.

Now let's add the decisions to the dataframe.

In [11]:
for i in range(len(df)):
    if df.iloc[i]['RSI']>70 and df.iloc[i]['MACD']<df.iloc[i]['Signal Line']:
        df.loc[i]['Decision']=-1
    elif df.iloc[i]['RSI']<30 and df.iloc[i]['MACD']>df.iloc[i]['Signal Line']:
        df.loc[i]['Decision']=1
    else:
        df.loc[i]['Decision']=0
        

In this part, we are going to create logs.
In our log, we'll keep;
- the Start Capital before the the long or short
- The Position Taken
- The StartPrice when we opened the position
- How Many of the Stock is bought
- At what time we opened the position
- When we closed the position
- The duration of the position
- The End Price when we closed the position
- The End Capital we gathered
- The +/-
- And the Perc on our Capital

In [12]:
position=0
startpos=0

initialmoney=1000

log = DataFrame(columns=['StartCapital','Position Taken','startprice','HowMany','StartTime','EndTime','Duration','endprice','EndCapital','ProfitLoss','Perc'],index=range(1000))
logcounter = 0

Now, we want to fill our log dataframe.
Since we don't know how the stock price will change at t+1 when we are at t, we will make the decision at time t. And long or short at t+1.
At first, this algorithm checks if there is an open position at time t. If not, it will take position at the time t+1 when the decision at time t is to open a position.

There are several assumptions that occur;
- If a position is open and the indicator shows that the decision is against the current position, close current at time t, open new position at time t+1.
- If the position gives more than %20 profit or loss is more than %20 close position.
- The position opening price is always changing and it has a random value close to t-1 price(%1 change at max)
- And if the indicator shows the same decision and time has passed more than 3 bars from the current open position, open the new position.
- Finally, it the stock data has come to an end, close position.

Using this algorithm we create our log.

In [13]:
for i in range(1,len(df)):
    if position!=0 and i!=len(df)-1:
            
        if df.iloc[i-1]['Decision']!=0:
            if logcounter==0:
                log.loc[logcounter]['StartCapital']=initialmoney
                
            log.loc[logcounter]['Position Taken']=position
            log.loc[logcounter]['startprice']=(df.iloc[i]['Prices']-df.iloc[i-1]['Prices'])*random.uniform(-1,1)+df.iloc[i-1]['Prices']
            log.loc[logcounter]['HowMany']=(log.loc[logcounter]['StartCapital']/df.iloc[i]['Prices'])*0.999
            log.loc[logcounter]['StartTime']=i
            startpos=i

        if position==1:
            if df.loc[i]['Decision']==-1 :
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=-1
            elif df.loc[i]['Decision']==1 and i-startpos+1>3:
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=1
            elif df.iloc[i]['Prices']>=log.iloc[logcounter]['startprice']*1.2 or df.iloc[i]['Prices']<=log.iloc[logcounter]['startprice']*0.8: 
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=0
                    
        elif position==-1:
            if df.loc[i]['Decision']==1 :
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=1
            elif df.loc[i]['Decision']==-1 and i-startpos+1>3 :
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=-1
            elif df.iloc[i]['Prices']<=log.iloc[logcounter]['startprice']*0.8 or df.iloc[i]['Prices']>=log.iloc[logcounter]['startprice']*1.2:
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1 
                position=0       
                    
    elif i==len(df)-1:
        if position==1:
            log.loc[logcounter]['EndTime']=i
            log.loc[logcounter]['Duration']=i-startpos+1
            log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
            log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
            log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
            log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
            position=0
        elif position==-1:
            log.loc[logcounter]['EndTime']=i
            log.loc[logcounter]['Duration']=i-startpos+1
            log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
            log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
            log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
            log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']

            position=0
    else:
        if df.loc[i]['Decision']==1:
            position=1
        elif df.loc[i]['Decision']==-1:
            position=-1
        else:
            position=0
            

log=log.dropna()
print(log)

   StartCapital Position Taken startprice   HowMany StartTime EndTime  \
0          1000             -1    1154.44  0.860707        53      70   
1       983.349             -1    1166.46  0.832561        71     125   
2       766.463             -1    1451.86  0.532974       138     681   
3       945.761              1    958.045  0.953986       707    1088   
4       1100.37             -1    1204.56  0.903326      1106    1502   
5       1300.17              1    973.012   1.33393      1515    1518   
6       1291.06              1    983.147   1.27385      1521    1607   
7          1507              1      979.1    1.5474      1832    1844   
8       1451.67              1    927.902   1.54634      1848    1939   
9       1687.46             -1    1116.11   1.50851      1945    1952   
10      1673.21             -1    1127.32   1.49491      1953    2237   
11      1330.92              1    1045.13   1.25604      2355    2600   
12      1279.16             -1    1027.69   1.26481

Now we want to check how accurate our decision are:

So we calculate the value:
- Opened long position and got profit
- Opened long position and lost money
- Opened short position and got profit
- Opened short position and lost profit

In [14]:
countlp=0
countln=0
countsp=0
countsn=0

for i in range(len(log)):
    if log.iloc[i]['Position Taken']==1 and log.iloc[i]['ProfitLoss']>0:
        countlp += 1
    elif log.iloc[i]['Position Taken']==1 and log.iloc[i]['ProfitLoss']<0:
        countln += 1
    elif log.iloc[i]['Position Taken']==-1 and log.iloc[i]['ProfitLoss']>0: 
        countsp += 1
    elif log.iloc[i]['Position Taken']==-1 and log.iloc[i]['ProfitLoss']<0:
        countsn += 1

df2=DataFrame(columns=['Total Count'],index=[-1,1])

df2['Total Count']=[countsp+countsn,countlp+countln]
df2['Correct Decision']=[countsp,countlp]
df2['Wrong Decision']=[countsn,countln]
df2['Accuracy']=[countlp/(countlp+countln),countsp/(countsp+countsn)]
df2.loc['Total']=[sum(df2['Total Count']),countlp+countsp,countln+countsn,(countlp+countsp)/(countlp+countsp+countln+countsn)]

print(df2)

       Total Count  Correct Decision  Wrong Decision  Accuracy
-1            32.0              11.0            21.0  0.437500
1             32.0              14.0            18.0  0.343750
Total         64.0              25.0            39.0  0.390625


In this part, we check how much money we put and how much we had at the end of the duration.

In [15]:
totallog = DataFrame(columns=['Initial Money','Profit or Loss','Perc'],index=range(1))

In [16]:
totallog['Initial Money']=initialmoney
totallog['Profit or Loss']=log.iloc[-1]['EndCapital']-initialmoney
totallog['Perc']=(log.iloc[-1]['EndCapital']-initialmoney)*100/initialmoney
print(totallog)

   Initial Money  Profit or Loss       Perc
0           1000      864.499363  86.449936


This creates only one example, for the system.
Normally all of these are given with functions (StockGenerator, MACDCalculator,RSICalculator,LogCreator, AccuracyControl).
However, since the project is explained in JupyterLab I only gave one example.

For the overall accuracy of the decision mechanism, we have to take many examples (10000-20000). Calculate our accuracy, calculate our profit ratio and fit these into confidence intervals of %80 (This is also an assumption. wanting more would probably make the system overfit, thus make us lose money). 

Since the decision mechanism is not well enough, preparation of analysis part and creating random charts are the important part at this time. 

I will probably try to use some Neural Network algorithms, different indicators, and data analysis on the effects of indicators to the stock price to create a more complex, more trustable decision mechanism.

Also the log creation part may be subject to changes, but the assumptions are generally valid at this point and only minor changes will be made.
